In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']
from sklearn.metrics import mean_squared_error
import random
import seaborn as sns
import math
from sklearn.preprocessing import MinMaxScaler
from models import *

In [2]:
def sliding_windows(x, y, seq_length):
    x_windows = []
    y_windows = []
    for i in range(seq_length, x.shape[0]):
        x_windows.append(x[i-seq_length:i, :])
        y_windows.append(y[i-seq_length:i+1])
    return np.array(x_windows), np.array(y_windows)

In [3]:
data = pd.read_csv('./assets/data.csv', index_col='Date')
data['y'] = data['Close']

data_x = data.iloc[:, :-1].values
data_y = data.iloc[:, -1].values

train_x, test_x = data_x[:-319, :], data_x[-319:, :]
train_y, test_y = data_y[:-319], data_y[-319:]

print(f'trainX: {train_x.shape} trainY: {train_y.shape}')
print(f'testX: {test_x.shape} testY: {test_y.shape}')

x_scaler = MinMaxScaler(feature_range = (0, 1))
y_scaler = MinMaxScaler(feature_range = (0, 1))

train_x = x_scaler.fit_transform(train_x)
train_y = y_scaler.fit_transform(train_y.reshape(-1, 1))
test_x = x_scaler.transform(test_x)
test_y = y_scaler.transform(test_y.reshape(-1, 1))

train_loader = DataLoader(TensorDataset(torch.from_numpy(train_x).float(), torch.from_numpy(train_y).float()),
                          batch_size=128, shuffle=True)

vae = VAE([train_x.shape[1], 400, 400, 400, 400, 10], 10, use_cuda=1)
num_epochs = 250
learning_rate = 0.0003
optimizer = torch.optim.Adam(vae.parameters(), lr=learning_rate)

print('Training the VAE model------------------')
history = np.zeros(num_epochs)
for epoch in range(num_epochs):
    total_loss = 0
    for x, _ in train_loader:
        x = x.to(vae.device)
        x_hat, z, mu, log_var = vae(x)
        loss = vae.criterion(x_hat, x, mu, log_var)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    history[epoch] = total_loss
    print(f'Epoch: {epoch + 1}/{num_epochs}, Loss: {total_loss:.4f}')

vae.eval()
with torch.no_grad():
    _, _, mu, _ = vae(torch.from_numpy(train_x).float().to(vae.device))
    mu = mu.cpu().numpy()

train_x = np.concatenate((train_x, mu), axis=1)

seq_length = 10
train_x, train_y = sliding_windows(train_x, train_y, seq_length)
print(f'train_x: {train_x.shape}, train_y: {train_y.shape}')

trainX: (2707, 57) trainY: (2707,)
testX: (319, 57) testY: (319,)
Training the VAE model------------------
Epoch: 1/250, Loss: 769.5591
Epoch: 2/250, Loss: 708.2346
Epoch: 3/250, Loss: 640.2910
Epoch: 4/250, Loss: 532.6404
Epoch: 5/250, Loss: 452.7827
Epoch: 6/250, Loss: 356.1686
Epoch: 7/250, Loss: 325.9710
Epoch: 8/250, Loss: 243.4519
Epoch: 9/250, Loss: 223.8246
Epoch: 10/250, Loss: 168.1114
Epoch: 11/250, Loss: 113.1002
Epoch: 12/250, Loss: 114.8276
Epoch: 13/250, Loss: 118.2852
Epoch: 14/250, Loss: 103.1270
Epoch: 15/250, Loss: 77.3466
Epoch: 16/250, Loss: 60.9898
Epoch: 17/250, Loss: 62.6590
Epoch: 18/250, Loss: 73.8302
Epoch: 19/250, Loss: 81.3866
Epoch: 20/250, Loss: 81.1956
Epoch: 21/250, Loss: 77.5761
Epoch: 22/250, Loss: 72.4586
Epoch: 23/250, Loss: 66.1455
Epoch: 24/250, Loss: 60.9336
Epoch: 25/250, Loss: 59.5793
Epoch: 26/250, Loss: 62.2905
Epoch: 27/250, Loss: 67.4894
Epoch: 28/250, Loss: 72.1275
Epoch: 29/250, Loss: 74.0820
Epoch: 30/250, Loss: 75.4756
Epoch: 31/250, Los

In [6]:
# Train the WGAN model and will fine-tune the model later
# Generate the data for training WGAN
train_wgan_data = DataLoader(TensorDataset(torch.from_numpy(train_x).float(), torch.from_numpy(train_y).float()), batch_size=128, shuffle=False)
wgan = WGAN(67, 10, 1)
num_epochs = 500
learning_rate = 0.000015
wgan.optimizers(lr=learning_rate)
wgan.training_step(num_epochs, train_wgan_data, seq_length)


[1/500] LossD: -0.00070 LossG:-0.39951
[2/500] LossD: -0.00220 LossG:-0.40650
[3/500] LossD: -0.00352 LossG:-0.41188


KeyboardInterrupt: 

In [ ]:
fake_data.shape

torch.Size([128, 11, 1])